In [ ]:
%load_ext autoreload
%autoreload 2
import os
project_name = "PaddleSeg"
os.chdir(os.path.join(os.getcwd().split(project_name, 1)[0], project_name))

In [ ]:
from utilities.dataset_manager import DatasetManager
from utilities.runner import Runner
root_directory = 'Z:\datasets\cracks\КЦТ_Фото_Шахты'
base_path = os.path.join('Z:', 'datasets', 'cracks', 'predict', 'hrsegnetb48')
dm = DatasetManager()
runner = Runner()

In [ ]:
extracted_subpaths = dm.copy_relevant_datasets(root_directory, base_path)

In [ ]:
for i, (path, working_dir, image_path) in enumerate(extracted_subpaths):
  runner.run_model(image_path, working_dir)

In [ ]:
working_dir = os.path.join(base_path,'selected', )
runner.run_model(os.path.join(working_dir, 'input'), working_dir)

In [ ]:
lucky_images = [
  '0_1',
  '0_125',
  '0_9',
  '0_32',
  '6_73',
  '7_1',
  '8_7',
  '8_12',
  '8_13',
  '8_21',
  '8_29',
  '8_12',
]


wrong_images = [
  '0_112',
  '0_119',
  '0_6',
  '0_11',
  '0_26',
  '0_67',
  '0_70',
  '0_73',
  '1_27',
  '10_10',
]

zero_images = [
  '0_118',
  '0_120',
  '0_24',
  '0_100',
  '1_4',
  '10_24',
  '10_33',
]

In [ ]:
dest_dir = os.path.join('Z:\datasets\cracks\predict\hrsegnetb48', 'selected', 'input')
selected_images = [fname.split('.')[0] for fname in os.listdir(dest_dir)]
selected_images

In [ ]:
import shutil
from PIL import Image
import os

def create_animated_gif(image_paths: list[str], output_path: str, duration: int=500):
    # Load images from the file paths
    images = [Image.open(path) for path in image_paths]

    # Ensure all images are converted to a consistent format
    converted_images = [image.convert('RGBA') for image in images]

    # Save as an animated GIF
    converted_images[0].save(output_path, save_all=True, append_images=converted_images[1:], duration=duration, loop=0, transparency=100)
    print(f"Animated GIF saved as {output_path}")

def find_file_with_extension(directory, name_start):
    # List all files in the given directory
    for file in os.listdir(directory):
        # Check if the file name starts with the specified string
        if file.startswith(name_start):
            # Extract the extension of the file
            _, file_extension = os.path.splitext(file)
            return file_extension

    return None

def extract(pref: str, fps: list[str], dest_dir: str, dir_name: str | None = None):
  gif_paths = []
  if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

  for file_prefix in fps:
    dir_name = dir_name or file_prefix.split('_')[0]
    ds_path = os.path.join(base_path, dir_name)
    
    input_dir = os.path.join(ds_path, 'input')
    print(ds_path, input_dir)
    input_extension = find_file_with_extension(input_dir, file_prefix)
    if input_extension is not None:
      input_path = os.path.join(input_dir, f'{file_prefix}{input_extension}')
      added_path = os.path.join(ds_path, 'added_prediction', f'{file_prefix}{input_extension}')
      pseudo_path = os.path.join(ds_path, 'pseudo_color_prediction', f'{file_prefix}.png')

      shutil.copy(input_path, os.path.join(dest_dir, f'{pref}_{file_prefix}_input{input_extension}'))
      shutil.copy(added_path, os.path.join(dest_dir, f'{pref}_{file_prefix}_added{input_extension}'))
      shutil.copy(pseudo_path, os.path.join(dest_dir, f'{pref}_{file_prefix}_pseudo.png'))
      gif_paths.append(input_path)
      gif_paths.append(added_path)
      gif_paths.append(pseudo_path)
      create_animated_gif([input_path, added_path, pseudo_path], os.path.join(dest_dir, f'{pref}_{file_prefix}.gif'), 2000)
    else:
       print('not found', file_prefix)
  
  # create_animated_gif(gif_paths, os.path.join(dest_dir, f'{pref}.gif'), 2000)

In [ ]:
dest_dir = os.path.join('Z:\datasets\cracks\predict\hrsegnetb48', 'extract')
# extract('zero', zero_images, dest_dir)
extract('selected', selected_images, dest_dir, dir_name='selected')

In [ ]:
concrete_images = [a.split('.')[0] for a in os.listdir(os.path.join(working_dir, 'input'))]
concrete_images

In [ ]:
dest_dir = os.path.join('Z:\datasets\cracks\predict\hrsegnetb48', 'extract')
extract('crack', concrete_images, dest_dir)

In [ ]:
import shutil

for fname in os.listdir(dest_dir):
  splat = fname.split('.')
  if len(splat) == 3:
    print(fname)

    shutil.move(os.path.join(dest_dir, fname), os.path.join(dest_dir, f'{splat[0]}.{splat[2]}'))
